In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import os
import chess.pgn
import numpy as np

In [ ]:
piece_values = {
    "p": 1,
    "n": 3,
    "b": 3,
    "r": 5,
    "q": 9,
    "k": 1000,
    "P": -1,
    "N": -3,
    "B": -3,
    "R": -5,
    "Q": -9,
    "K": -1000,
}

def createStateObj(board):
    # convert state into a 8x8x12 tensor
    state = np.zeros(774) # 8 * 8 * 12 + 5
    net_piece_value = 0

    for square, piece in board.piece_map().items():
        for square, piece in board.piece_map().items():
            net_piece_value += piece_values[str(piece)]
            if piece.color == chess.WHITE:
                state[square * 12 + piece.piece_type - 1] = 1
            else:
                state[square * 12 + piece.piece_type + 5] = 1

    # flatten state
    state = state.flatten()

    # append the 5 states above
    p1_can_castle_queenside = board.has_queenside_castling_rights(chess.WHITE)
    p1_can_castle_kingside = board.has_kingside_castling_rights(chess.WHITE)
    p2_can_castle_queenside = board.has_queenside_castling_rights(chess.BLACK)
    p2_can_castle_kingside = board.has_kingside_castling_rights(chess.BLACK)
    turn = board.turn

    state[768] = p1_can_castle_queenside
    state[769] = p1_can_castle_kingside
    state[770] = p2_can_castle_queenside
    state[771] = p2_can_castle_kingside
    state[772] = net_piece_value
    state[773] = float(turn)
    
    return state


start = 0
flag = 0

def createData(start, n_data=10000):
    with open("DATABASE4U_CLEANED.pgn", "r") as rf:
        for i in range(start):
            game = chess.pgn.read_game(rf)

        X = []
        y = []

        for i in range(start, start + n_data):
            game = chess.pgn.read_game(rf)
            if game is None:
                flag = 1
                break

            if game.headers["Result"] == '1/2-1/2':
                win = 0
            elif game.headers["Result"] == '1-0':
                win = 1
            elif game.headers["Result"] == '0-1':
                win = -1
            else:
                print("Error: Unexpected result string" + game.headers["Result"])
                continue

            board = game.board()

            lst = list(game.mainline_moves())
            l_lst = len(lst)
            r1, r2 = random.randint(0, l_lst), random.randint(0, l_lst)
            r1, r2 = min(r1, r2), max(r1, r2)

            for i in range(r1):
                board.push(lst[i])
            X.append(createStateObj(board))
            y.append(win)

            if r1 == r2:
                continue

            for i in range(r1, r2):
                board.push(lst[i])
            X.append(createStateObj(board))
            y.append(win)
    
    data = list(zip(X, y))
    random.shuffle(data)
    X, y = zip(*data)

    return X, y

In [ ]:
class MyNetwork(nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__()
        self.fc1 = nn.Linear(774, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 32)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [ ]:
# hyperparameters
lr = 0.0001
num_epochs = 100

In [ ]:
class DataLoader:
    def __init__(self, start=0):
        self.start = start

    def _create_data(self, step):
        if os.path.exists(f"data/{self.start}_{self.start+step}X.npy") and os.path.exists(
            f"data/{self.start}_{self.start+step}y.npy"
        ):
            X = np.load(f"data/{self.start}_{self.start+step}X.npy")
            y = np.load(f"data/{self.start}_{self.start+step}y.npy")

        else:
            X, y = createData(self.start, step)
            np.save(f"data/{self.start}_{self.start+step}X.npy", X)
            np.save(f"data/{self.start}_{self.start+step}y.npy", y)

        return X, y

    def get_data(self, step=10000):
        X, y = self._create_data(step)
        self.start += step

        return list(zip(X, y))


trainLoader = DataLoader()

In [ ]:
net = MyNetwork()

# Define the optimizer
optimizer = optim.Adam(net.parameters(), lr=lr)

# Define the loss function
criterion = nn.MSELoss()

# Train the network
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1} of {num_epochs}")
    running_loss = 0.0
    for i, data in enumerate(trainLoader.get_data()):
        # Get the inputs
        inputs, labels = data

        inputs = torch.tensor(inputs, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.float32)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
    
    # save model into an .h5 file every 10 epochs
    if epoch % 10 == 9:
        torch.save(net.state_dict(), f"model_{epoch}.h5")

In [ ]:
trainLoader = DataLoader(0)
a = trainLoader.get_data(10)

for i, data in enumerate(a):
    # Get the inputs
    inputs, labels = data

    # convert inputs and labels into torch tensors
    inputs = torch.tensor(inputs, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.float32)

    # Zero the parameter gradients
    optimizer.zero_grad()

    # Forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # Print statistics
    running_loss += loss.item()
    if i % 100 == 99:
        print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / 100))
        running_loss = 0.0

# save model into an .h5 file every 10 epochs
if epoch % 10 == 9:
    torch.save(net.state_dict(), f"model_{epoch}.h5")